In [ ]:
###Helper functions for image preprocessing
impor cv2
import numpy as np

# Helper functions
def load_image(image_path):
    return cv2.imread(image_path)

# Function to create a saturation mask with a given threshold
def generate_saturation_mask(image, threshold=50):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    s_channel = hsv[:, :, 1]
    return cv2.inRange(s_channel, threshold, 255)

# Function to apply a saturation mask to an image
def apply_saturation_mask(image, saturation_mask):
    return cv2.bitwise_and(image, image, mask=saturation_mask)

# Function to create a color mask that can wrap around the hue spectrum
def create_wrapped_color_mask(hsv_image, lower_hue, upper_hue):
    if lower_hue > upper_hue:
        lower_bound1 = np.array([lower_hue, 50, 50])
        upper_bound1 = np.array([180, 255, 255])
        lower_bound2 = np.array([0, 50, 50])
        upper_bound2 = np.array([upper_hue, 255, 255])
        mask1 = cv2.inRange(hsv_image, lower_bound1, upper_bound1)
        mask2 = cv2.inRange(hsv_image, lower_bound2, upper_bound2)
        mask = cv2.bitwise_or(mask1, mask2)
    else:
        lower_bound = np.array([lower_hue, 50, 50])
        upper_bound = np.array([upper_hue, 255, 255])
        mask = cv2.inRange(hsv_image, lower_bound, upper_bound)
    return mask

# Function to cluster hue values that are close to each other
def cluster_hues(hues, threshold=10):
    clusters = []
    hues_sorted = sorted(hues)
    cluster = [hues_sorted[0]]
    for hue in hues_sorted[1:]:
        if abs(hue - cluster[-1]) <= threshold:
            cluster.append(hue)
        else:
            clusters.append(cluster)
            cluster = [hue]
    clusters.append(cluster)
    return clusters

# Function to apply median filter to a mask
def apply_median_filter(mask):
    return cv2.medianBlur(mask, 7)

# Function to validate a mask based on the white area (non-zero pixels)
def validate_mask(mask, area_threshold=500):
    white_area = cv2.countNonZero(mask)
    return white_area > area_threshold

# Function to create filtered and clustered color masks based on the saturation threshold and other parameters
def create_filtered_clustered_color_masks(image, saturation_threshold=60, hue_peak_detection_threshold=0.05, hue_threshold=10, area_threshold=500):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    saturation = hsv[:, :, 1]
    bright_colors_mask = cv2.inRange(saturation, saturation_threshold, 255)
    hue_channel = hsv[:, :, 0][bright_colors_mask > 0]
    hist, _ = np.histogram(hue_channel, bins=180, range=(0, 180))
    hist_normalized = hist / np.max(hist)
    prominent_hues = np.where(hist_normalized > hue_peak_detection_threshold)[0]
    hue_clusters = cluster_hues(prominent_hues, hue_threshold)
    filtered_color_masks = {}
    for cluster in hue_clusters:
        lower_hue = min(cluster) - 10
        upper_hue = max(cluster) + 10
        mask = create_wrapped_color_mask(hsv, lower_hue, upper_hue)
        filtered_mask = apply_median_filter(mask)
        if validate_mask(filtered_mask, area_threshold):
            filtered_color_masks[f'hue_{lower_hue}-{upper_hue}'] = filtered_mask
    return filtered_color_masks

def augment_saturation_mask(saturation_mask, hue_mask):
    combined_mask = cv2.bitwise_or(saturation_mask, hue_mask)
    return combined_mask

def apply_combined_mask(image, combined_mask):
    masked_image_with_combined_mask = cv2.bitwise_and(image, image, mask=combined_mask)
    return masked_image_with_combined_mask

def generate_gray_mask(image, lower_threshold=120, upper_threshold=150, kernel_size=5):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv_image[:,:,2], lower_threshold, upper_threshold)
    kernel = np.ones((kernel_size, kernel_size), np.uint8)
    refined_mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    refined_mask = cv2.morphologyEx(refined_mask, cv2.MORPH_CLOSE, kernel)
    return refined_mask

def apply_gray_mask(image, gray_mask):
    masked_image_with_gray_mask = cv2.bitwise_and(image, image, mask=cv2.bitwise_not(gray_mask))
    return masked_image_with_gray_mask

def calculate_writing_ratio(mask, edge_threshold=150):
    edges = cv2.Canny(mask, edge_threshold, edge_threshold * 2)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    total_contour_area = sum(cv2.contourArea(contour) for contour in contours)
    edge_pixels_within_contours = 0
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        edge_pixels_within_contours += np.sum(edges[y:y+h, x:x+w])
    if total_contour_area > 0:
        feature_proportion_within_contours = edge_pixels_within_contours / total_contour_area
    else:
        feature_proportion_within_contours = 0
    return feature_proportion_within_contours

def create_generalized_color_masks(image, saturation_threshold=60, hue_peak_detection_threshold=0.05):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    saturation = hsv[:, :, 1]
    bright_colors_mask = cv2.inRange(saturation, saturation_threshold, 255)
    hue_channel = hsv[:, :, 0][bright_colors_mask > 0]
    hist, _ = np.histogram(hue_channel, bins=180, range=(0, 180))
    hist_normalized = hist / np.max(hist)
    prominent_hues = np.where(hist_normalized > hue_peak_detection_threshold)[0]
    color_masks = {}
    for hue in prominent_hues:
        mask = create_wrapped_color_mask(hsv, hue - 10, hue + 10)
        color_masks[f'hue_{hue}'] = mask
    return color_masks

def create_wrapped_color_mask(hsv_image, lower_hue, upper_hue):
    if lower_hue > upper_hue:
        lower_bound1 = np.array([lower_hue, 50, 50])
        upper_bound1 = np.array([180, 255, 255])
        lower_bound2 = np.array([0, 50, 50])
        upper_bound2 = np.array([upper_hue, 255, 255])
        mask1 = cv2.inRange(hsv_image, lower_bound1, upper_bound1)
        mask2 = cv2.inRange(hsv_image, lower_bound2, upper_bound2)
        mask = cv2.bitwise_or(mask1, mask2)
    else:
        lower_bound = np.array([lower_hue, 50, 50])
        upper_bound = np.array([upper_hue, 255, 255])
        mask = cv2.inRange(hsv_image, lower_bound, upper_bound)
    return mask

def create_writing_mask(preprocessed_image, color_masks, writing_ratio_threshold=60):
    writing_mask = np.zeros_like(preprocessed_image[:, :, 0])
    for hue, mask in color_masks.items():
        writing_ratio = calculate_writing_ratio(mask)
        if writing_ratio > writing_ratio_threshold:
            writing_mask = cv2.bitwise_or(writing_mask, mask)
    return writing_mask

def apply_writing_mask(image, writing_mask):
    masked_image_with_writing_mask = cv2.bitwise_and(image, image, mask=cv2.bitwise_not(writing_mask))
    return masked_image_with_writing_mask

# New helper functions for additional preprocessing
def remove_grayish_pixels(image):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_gray = np.array([0, 0, 40])  # Lower bound for gray colors
    upper_gray = np.array([180, 65, 210])  # Upper bound for gray colors
    gray_mask = cv2.inRange(hsv_image, lower_gray, upper_gray)
    gray_mask_inv = cv2.bitwise_not(gray_mask)
    gray_mask_inv_3channel = cv2.cvtColor(gray_mask_inv, cv2.COLOR_GRAY2BGR)
    return cv2.bitwise_and(image, gray_mask_inv_3channel)

def apply_thresholding(gray_image):
    _, thresholded_image = cv2.threshold(gray_image, 15, 255, cv2.THRESH_BINARY)
    return thresholded_image

def apply_median_filter(image):
    return cv2.medianBlur(image, 7)

def mask_original_image(original_image, mask_image):
    if len(mask_image.shape) == 2:
        binary_mask_3channel = cv2.cvtColor(mask_image, cv2.COLOR_GRAY2BGR)
    else:
        binary_mask_3channel = mask_image
    return cv2.bitwise_and(original_image, binary_mask_3channel)

# Modified pipeline function including the additional preprocessing steps

def preprocess_image(image_path):
    # Load the image
    original_image = load_image(image_path)

    # Preprocess the image using saturation mask with a threshold of 50
    saturation_mask = generate_saturation_mask(original_image, threshold=50)
    preprocessed_image = apply_saturation_mask(original_image, saturation_mask)

    # Create filtered clustered color masks instead of generalized color masks
    color_masks = create_filtered_clustered_color_masks(original_image, saturation_threshold=50)


    # Find a specific hue mask (e.g., hue 80) and augment the saturation mask
    hue_80_mask = color_masks.get('hue_80', None)
    if hue_80_mask is not None:
        combined_mask = augment_saturation_mask(saturation_mask, hue_80_mask)
    else:
        combined_mask = saturation_mask

    # Apply the combined mask to the image
    image_with_combined_mask_applied = apply_combined_mask(original_image, combined_mask)

    # Generate and apply gray mask to the image
    gray_mask = generate_gray_mask(image_with_combined_mask_applied)
    image_with_gray_mask_applied = apply_gray_mask(image_with_combined_mask_applied, gray_mask)

    # Create and apply writing mask
    writing_mask = create_writing_mask(image_with_gray_mask_applied, color_masks)
    image_with_writing_mask_applied = apply_writing_mask(image_with_gray_mask_applied, writing_mask)

    # Additional preprocessing to remove grayish pixels
    no_gray_image = remove_grayish_pixels(image_with_writing_mask_applied)
    gray_image = cv2.cvtColor(no_gray_image, cv2.COLOR_BGR2GRAY)
    thresholded_image = apply_thresholding(gray_image)
    filtered_image = apply_median_filter(thresholded_image)

    # Apply the final mask to the original image
    final_image = mask_original_image(original_image, filtered_image)
    
    return final_image

# Example usage:
# processed_image = process_sticky_notes_pipeline('/path/to/your/image.jpg')
# cv2.imwrite('/path/to/save/processed_image.jpg', processed_image)


In [1]:
##Helper functions for extracting contours from preprocessed image

import cv2
import numpy as np

# Helper functions for color mask creation
def create_wrapped_color_mask(hsv_image, lower_hue, upper_hue):
    if lower_hue > upper_hue:
        lower_bound1 = np.array([lower_hue, 50, 50])
        upper_bound1 = np.array([180, 255, 255])
        lower_bound2 = np.array([0, 50, 50])
        upper_bound2 = np.array([upper_hue, 255, 255])
        mask1 = cv2.inRange(hsv_image, lower_bound1, upper_bound1)
        mask2 = cv2.inRange(hsv_image, lower_bound2, upper_bound2)
        mask = cv2.bitwise_or(mask1, mask2)
    else:
        lower_bound = np.array([lower_hue, 50, 50])
        upper_bound = np.array([upper_hue, 255, 255])
        mask = cv2.inRange(hsv_image, lower_bound, upper_bound)
    return mask

def cluster_hues(hues, threshold=10):
    clusters = []
    hues_sorted = sorted(hues)
    cluster = [hues_sorted[0]]
    for hue in hues_sorted[1:]:
        if abs(hue - cluster[-1]) <= threshold:
            cluster.append(hue)
        else:
            clusters.append(cluster)
            cluster = [hue]
    clusters.append(cluster)
    return clusters

def apply_median_filter(mask):
    return cv2.medianBlur(mask, 7)

def validate_mask(mask, area_threshold=500):
    white_area = cv2.countNonZero(mask)
    return white_area > area_threshold

def create_filtered_clustered_color_masks(image, saturation_threshold=60, hue_peak_detection_threshold=0.05, hue_threshold=10, area_threshold=500):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    saturation = hsv[:, :, 1]
    bright_colors_mask = cv2.inRange(saturation, saturation_threshold, 255)
    hue_channel = hsv[:, :, 0][bright_colors_mask > 0]
    hist, _ = np.histogram(hue_channel, bins=180, range=(0, 180))
    hist_normalized = hist / np.max(hist)
    prominent_hues = np.where(hist_normalized > hue_peak_detection_threshold)[0]
    hue_clusters = cluster_hues(prominent_hues, hue_threshold)
    filtered_color_masks = {}
    for cluster in hue_clusters:
        lower_hue = min(cluster) - 10
        upper_hue = max(cluster) + 10
        mask = create_wrapped_color_mask(hsv, lower_hue, upper_hue)
        filtered_mask = apply_median_filter(mask)
        if validate_mask(filtered_mask, area_threshold):
            filtered_color_masks[f'hue_{lower_hue}-{upper_hue}'] = filtered_mask
    return filtered_color_masks

# Contour detection and processing functions
def identify_valid_contours(gray_img, min_size=75):
    adaptive_thresh = cv2.adaptiveThreshold(gray_img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                            cv2.THRESH_BINARY_INV, 11, 1)
    kernel = np.ones((5, 5), np.uint8)
    morphed = cv2.morphologyEx(adaptive_thresh, cv2.MORPH_CLOSE, kernel)
    contours, _ = cv2.findContours(morphed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    valid_contours = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if w >= min_size and h >= min_size:
            valid_contours.append(contour)
    return valid_contours


import cv2
import numpy as np

def modified_find_most_square_contour(contours, min_size=75):
    closest_to_square = None
    closest_contour = None
    min_aspect_ratio_deviation = float('inf')
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if w < min_size or h < min_size:
            continue  # Skip contours that are too small
        aspect_ratio = float(w) / h
        if 0.75 <= aspect_ratio <= 1.33:  # Adjusted aspect ratio check
            deviation = abs(aspect_ratio - 1)
            if deviation < min_aspect_ratio_deviation:
                min_aspect_ratio_deviation = deviation
                closest_to_square = (x, y, w, h)
                closest_contour = contour
    if closest_contour is not None:
        typical_sticky_note_dimension = (w, h)
        isolated_sticky_notes = [closest_contour]
    else:
        typical_sticky_note_dimension = None
        isolated_sticky_notes = []
    return closest_to_square, closest_contour, isolated_sticky_notes


def is_approximately_square(contour, side_length):
    epsilon = 0.04 * cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, epsilon, True)
    if len(approx) != 4:
        return False
    for i in range(4):
        v1 = approx[i] - approx[i-1]
        v2 = approx[i] - approx[(i+1)%4]
        angle = np.arccos(np.dot(v1.ravel(), v2.ravel()) / (np.linalg.norm(v1) * np.linalg.norm(v2))) * (180/np.pi)
        if not 75 < angle < 105:
            return False
    for i in range(4):
        side = np.linalg.norm(approx[i] - approx[i-1])
        if not 0.7 * side_length < side < 1.3 * side_length:
            return False
    return True

def process_remaining_sticky_notes(contours, isolated_sticky_notes):
    complex_contours = []
    for contour in contours:
        if isolated_sticky_notes:  # Check if there are any isolated sticky notes
            _, _, w, h = cv2.boundingRect(isolated_sticky_notes[0])
            if is_approximately_square(contour, max(w, h)):
                isolated_sticky_notes.append(contour)
            else:
                complex_contours.append(contour)
        else:
            complex_contours.append(contour)  # If no isolated sticky notes, consider contour as complex
    return isolated_sticky_notes, complex_contours

def detect_and_categorize_contours(preprocessed_img):
    valid_contours = identify_valid_contours(preprocessed_img)
    
    # Find the most square contour which is assumed to be a typical sticky note
    _, _, isolated_sticky_notes = modified_find_most_square_contour(valid_contours)
    
    # Process remaining contours to categorize them
    isolated_sticky_notes, complex_contours = process_remaining_sticky_notes(valid_contours, isolated_sticky_notes)
    
    # Calculate average sticky note dimension
    if isolated_sticky_notes:
        average_dimension = np.mean([cv2.boundingRect(contour)[2:4] for contour in isolated_sticky_notes], axis=0)
    else:
        average_dimension = (0, 0)

    return isolated_sticky_notes, complex_contours, average_dimension

def calculate_angle(pt1, pt2, pt0):
    dx1 = pt1[0][0] - pt0[0][0]
    dy1 = pt1[0][1] - pt0[0][1]
    dx2 = pt2[0][0] - pt0[0][0]
    dy2 = pt2[0][1] - pt0[0][1]
    inner_product = dx1 * dx2 + dy1 * dy2
    len1 = np.sqrt(dx1 * dx1 + dy1 * dy1)
    len2 = np.sqrt(dx2 * dx2 + dy2 * dy2)
    angle = np.arccos(inner_product / (len1 * len2 + 1e-10))
    return angle

def calculate_corner_type(pt1, pt2, pt0):
    vector_a = np.array(pt1) - np.array(pt0)
    vector_b = np.array(pt2) - np.array(pt0)
    angle = np.math.atan2(np.linalg.det([vector_a, vector_b]), np.dot(vector_a, vector_b))
    return angle

def refine_epsilon_light(contour, initial_epsilon_factor=0.01, refinement_factor=0.7):
    epsilon = initial_epsilon_factor * cv2.arcLength(contour, True)
    new_epsilon = epsilon * refinement_factor
    new_approx = cv2.approxPolyDP(contour, new_epsilon, True)
    return new_approx

def distinguish_inside_outside_corners(contours, epsilon_factor=0.01):
    inside_corners = []
    outside_corners = []
    for contour in contours:
        approx = refine_epsilon_light(contour, epsilon_factor)
        for i in range(len(approx)):
            pt0 = approx[i][0]
            pt1 = approx[(i - 1) % len(approx)][0]
            pt2 = approx[(i + 1) % len(approx)][0]
            angle = calculate_corner_type(pt1, pt2, pt0)
            if angle < 0:
                inside_corners.append(pt0)
            else:
                outside_corners.append(pt0)
    return inside_corners, outside_corners

def traverse_contour_and_list_corners(start_corner, all_corners, all_contours):
    ordered_corners = []
    for contour in all_contours:
        try:
            start_index = next(i for i, c in enumerate(contour) if (c[0] == start_corner).all())
            contour_sequence = np.roll(contour, -start_index, axis=0)
            for point in contour_sequence:
                point = point[0]
                for original_corner, designation in all_corners:
                    if (point == original_corner).all():
                        ordered_corners.append((point, designation))
                        break
            if len(ordered_corners) > 1 and (ordered_corners[-1][0] == start_corner).all():
                return ordered_corners[:-1]
        except StopIteration:
            continue
    return ordered_corners

def generate_corner_triplets(ordered_corners):
    triplets = []
    length = len(ordered_corners)
    for i in range(length):
        corner1 = ordered_corners[i]
        corner2 = ordered_corners[(i + 1) % length]
        corner3 = ordered_corners[(i + 2) % length]
        if corner1[1] == 'outside' and corner2[1] == 'outside' and corner3[1] == 'outside':
            triplets.append((corner1[0], corner2[0], corner3[0]))
    return triplets

def verify_corner_triplets_with_tolerance(triplets, avg_sticky_note_dimension, tolerance=0.30):
    verified_triplets = []
    avg_note_size = np.mean(avg_sticky_note_dimension)
    for triplet in triplets:
        couched_corner = triplet[1]
        other_corners = [triplet[0], triplet[2]]
        distances = [np.linalg.norm(np.array(couched_corner) - np.array(corner)) for corner in other_corners]
        within_tolerance = all(avg_note_size * (1 - tolerance) <= d <= avg_note_size * (1 + tolerance) for d in distances)
        if within_tolerance:
            verified_triplets.append((triplet, within_tolerance))
    return verified_triplets

def create_new_contours_from_triplets(triplets):
    new_contours = []
    for triplet in triplets:
        couched_corner = triplet[1]
        outer_corners = [triplet[0], triplet[2]]
        perp_points = [calculate_perpendicular_point(couched_corner, corner) for corner in outer_corners]
        intersection_point = find_intersection((outer_corners[0], perp_points[0]), (outer_corners[1], perp_points[1]))
        new_contour = [couched_corner, outer_corners[0], intersection_point, outer_corners[1]]
        new_contours.append(new_contour)
    return new_contours

def calculate_perpendicular_point(corner1, corner2):
    dx = corner2[0] - corner1[0]
    dy = corner2[1] - corner1[1]
    length = np.sqrt(dx**2 + dy**2)
    perp_x = corner2[0] - dy * (length / np.sqrt(dx**2 + dy**2))
    perp_y = corner2[1] + dx * (length / np.sqrt(dx**2 + dy**2))
    return int(perp_x), int(perp_y)

def find_intersection(line1, line2):
    x1, y1, x2, y2 = line1[0][0], line1[0][1], line1[1][0], line1[1][1]
    x3, y3, x4, y4 = line2[0][0], line2[0][1], line2[1][0], line2[1][1]
    denom = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)
    if denom == 0:
        return None
    intersect_x = ((x1 * y2 - y1 * x2) * (x3 - x4) - (x1 - x2) * (x3 * y4 - y3 * x4)) / denom
    intersect_y = ((x1 * y2 - y1 * x2) * (y3 - y4) - (y1 - y2) * (x3 * y4 - y3 * x4)) / denom
    return int(intersect_x), int(intersect_y)

def exclude_enclosed_corners(all_corners, identified_contours):
    def reformat_point_for_cv(point):
        return (int(point[0]), int(point[1]))

    excluded_corners = []
    for corner in all_corners:
        is_enclosed = False
        corner_formatted = reformat_point_for_cv(corner)
        for sticky_note_contour in identified_contours:
            if cv2.pointPolygonTest(sticky_note_contour, corner_formatted, False) >= 0:
                is_enclosed = True
                break
        if not is_enclosed:
            excluded_corners.append(corner)
    return excluded_corners

def find_bounding_box(corners):
    x_coords = [pt[0] for pt in corners]
    y_coords = [pt[1] for pt in corners]
    return min(x_coords), min(y_coords), max(x_coords), max(y_coords)

def calculate_contour_area_within_box(contour, box, image_shape):
    mask = np.zeros(image_shape, dtype=np.uint8)
    cv2.drawContours(mask, [contour], -1, 255, -1)
    x_min, y_min, x_max, y_max = box
    return np.sum(mask[y_min:y_max, x_min:x_max]) // 255

def center_sticky_note_in_box(box, sticky_note_size):
    x_min, y_min, x_max, y_max = box
    box_center = ((x_min + x_max) // 2, (y_min + y_max) // 2)
    half_size = sticky_note_size // 2
    return [(box_center[0] - half_size, box_center[1] - half_size), 
            (box_center[0] + half_size, box_center[1] - half_size),
            (box_center[0] + half_size, box_center[1] + half_size), 
            (box_center[0] - half_size, box_center[1] + half_size)]

def sticky_note_detection_pipeline(complex_contour, inside_corners, outside_corners, identified_sticky_notes, image_shape, avg_sticky_note_side_length=186):
    # Combining inside and outside corners
    all_corners = inside_corners + outside_corners
    excluded_corners = exclude_enclosed_corners(all_corners, identified_sticky_notes)

    potential_sticky_notes = []
    bounding_box = find_bounding_box(excluded_corners)

    # Check if each of the bounding box dimensions is at least 75% of the average sticky note length
    bounding_box_width = bounding_box[2] - bounding_box[0]
    bounding_box_height = bounding_box[3] - bounding_box[1]
    if bounding_box_width >= avg_sticky_note_side_length * 0.75 and bounding_box_height >= avg_sticky_note_side_length * 0.75:
        # Create a centered sticky note contour
        potential_sticky_note_contour = center_sticky_note_in_box(bounding_box, avg_sticky_note_side_length)
        potential_sticky_note_contour = np.array(potential_sticky_note_contour, dtype=np.int32).reshape(-1, 1, 2)
        potential_sticky_notes.append(potential_sticky_note_contour)

    return potential_sticky_notes

def process_complex_contours(complex_contours, image_shape, avg_sticky_note_dimension=(186, 186), tolerance=0.30):
    all_sticky_notes = []

    for contour in complex_contours:
        # First pass to find inside and outside corners and create the initial sticky note contours
        inside_corners, outside_corners = distinguish_inside_outside_corners([contour])
        combined_corners = [(corner, 'inside') for corner in inside_corners] + \
                           [(corner, 'outside') for corner in outside_corners]

        if combined_corners:
            # Traverse the contour and list corners to get them in order
            start_corner = combined_corners[0][0]
            ordered_corners = traverse_contour_and_list_corners(start_corner, combined_corners, [contour])
            # Generate corner triplets from the ordered corners
            corner_triplets = generate_corner_triplets(ordered_corners)
            # Verify the corner triplets with the given tolerance
            verified_triplets = verify_corner_triplets_with_tolerance(corner_triplets, avg_sticky_note_dimension, tolerance)
            # Create new contours from the verified triplets
            new_contours = create_new_contours_from_triplets([triplet for triplet, verified in verified_triplets if verified])
            # Extend the sticky notes with new contours
            sticky_notes = [np.array(contour, dtype=np.int32).reshape((-1, 1, 2)) for contour in new_contours]
            all_sticky_notes.extend(sticky_notes)

        # Additional detection phase within complex contours
        additional_sticky_notes = sticky_note_detection_pipeline(contour, inside_corners, outside_corners, all_sticky_notes, image_shape, avg_sticky_note_dimension[0])
        # Add any new detected sticky notes to the list
        all_sticky_notes.extend(additional_sticky_notes)

    return all_sticky_notes

def extract_contours(image):
    filtered_color_masks = create_filtered_clustered_color_masks(image)
    all_isolated_sticky_notes = []
    all_complex_contours = []
    dimensions_list = []

    # Process each color mask
    for color_range, mask in filtered_color_masks.items():
        # Ensure the mask is in the correct format for contour detection (single-channel)
        if len(mask.shape) == 3 and mask.shape[2] == 3:  # If the mask is not a single channel, convert it
            gray_mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        else:
            gray_mask = mask

        # Corrected unpacking here
        isolated_sticky_notes, complex_contours, average_dimension = detect_and_categorize_contours(gray_mask)
        all_isolated_sticky_notes.extend(isolated_sticky_notes)
        all_complex_contours.extend(complex_contours)
        if average_dimension is not None:  # Corrected comparison here
            dimensions_list.append(average_dimension)


    # Process complex contours to identify additional sticky notes
    image_shape = image.shape[:2]
    additional_sticky_notes = process_complex_contours(all_complex_contours, image_shape, average_dimension)

    # Combine isolated sticky notes with those extracted from complex contours
    all_sticky_notes = all_isolated_sticky_notes + additional_sticky_notes

    return all_sticky_notes

In [2]:
#Helper functions for extracting images from note contours

import cv2
import numpy as np
from skimage import color
from matplotlib.colors import rgb2hex
import os
import zipfile

def order_points(pts):
    rect = np.zeros((4, 2), dtype="float32")
    s = pts.sum(axis=1)
    diff = np.diff(pts, axis=1)
    rect[0] = pts[np.argmin(s)]
    rect[1] = pts[np.argmin(diff)]
    rect[2] = pts[np.argmax(s)]
    rect[3] = pts[np.argmax(diff)]
    return rect

def four_point_transform(image, pts):
    rect = order_points(pts)
    (tl, tr, br, bl) = rect
    widthA = np.linalg.norm(br - bl)
    widthB = np.linalg.norm(tr - tl)
    maxWidth = max(int(widthA), int(widthB))
    heightA = np.linalg.norm(tr - br)
    heightB = np.linalg.norm(tl - bl)
    maxHeight = max(int(heightA), int(heightB))
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]
    ], dtype="float32")
    M = cv2.getPerspectiveTransform(rect, dst)
    return cv2.warpPerspective(image, M, (maxWidth, maxHeight))

def get_average_color(image):
    avg_color_per_row = np.mean(image, axis=0)
    avg_color = np.mean(avg_color_per_row, axis=0)
    return avg_color

def rgb_to_hex(rgb):
    return "#{:02x}{:02x}{:02x}".format(int(rgb[0]), int(rgb[1]), int(rgb[2]))

def find_contour_center(contour):
    M = cv2.moments(contour)
    if M["m00"] != 0:
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
    else:
        cX, cY = contour[0][0]
    return cX, cY

def process_image_into_notes(image_path, contours):
    image = cv2.imread(image_path)
    
    # Extracting the base name of the image file (e.g., 'IMG_0220' from 'IMG_0220.jpg')
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    
    # Constructing the zip file name with the image name prefix and '_notes.zip' suffix
    zip_filename = os.path.join(os.getcwd(), f"{base_name}_notes.zip")

    with zipfile.ZipFile(zip_filename, 'w') as zipf:
        for i, c in enumerate(contours):
            rect = cv2.minAreaRect(c)
            box = cv2.boxPoints(rect)
            processed_contour = np.int0(box)
            angle = rect[2]
            if angle < -45:
                angle += 90

            warped_image = four_point_transform(image, processed_contour.reshape(4, 2))
            avg_color = get_average_color(warped_image)
            hex_color = rgb_to_hex(avg_color)
            center_x, center_y = find_contour_center(c)

            # Encoding the image and writing directly to the zip file
            _, buffer = cv2.imencode('.png', warped_image)
            image_name = f"{base_name}_image-{i}_{center_x}_{center_y}_{int(angle)}_{hex_color.replace('#', '')}.png"
            zipf.writestr(image_name, buffer.tobytes())

    return zip_filename

In [ ]:
#Main workchain function for processing an image into sticky notes

import sys
import os
import zipfile

# Importing functions from the helper files using the convention
import HELPER_preprocess_image as preproc
import HELPER_extract_contours as extract
import HELPER_extract_images_from_note_contours as process_notes

# Function to process the entire poster image and extract notes
def process_sticky_note_poster(image_path):
    final_preprocessed_image = preproc.preprocess_image(image_path)
    sticky_note_contours = extract.extract_contours(final_preprocessed_image)
    notes_zip_file = process_notes.process_image_into_notes(image_path, sticky_note_contours)
    return notes_zip_file

# Main execution
if __name__ == "__main__":
    # Check if an image path was provided
    if len(sys.argv) != 2:
        print("Usage: python process_poster.py <image_path>")
        sys.exit(1)
    
    image_path = sys.argv[1]
    
    # Check if the provided path is valid
    if not os.path.isfile(image_path):
        print(f"The provided image path does not exist: {image_path}")
        sys.exit(1)
    
    # Process the image and get the path to the saved zip file
    zip_file_path = process_sticky_note_poster(image_path)
    
    # Save the zip file in the current working directory
    local_zip_path = os.path.join(os.getcwd(), os.path.basename(zip_file_path))
    os.rename(zip_file_path, local_zip_path)
    
    print(f"Process complete. Zip file saved to: {local_zip_path}")


In [ ]:
#Main function for generating a Miro board from a set of images

import csv
import requests
import sys
import os

# Constants for sticky note dimensions
NOTE_WIDTH = 372  # Width of the sticky note
NOTE_HEIGHT = 372  # Height of the sticky note

# Function to clear the console
def clear():
    if sys.platform == 'darwin':
        os.system('clear')
    else:
        os.system('cls')

# Function to read Miro API Key, Client ID, Client Secret, and Board ID from a text file
def read_miro_credentials(file_path='miro_api_key.txt'):
    credentials = {}
    try:
        with open(file_path, 'r') as file:
            for line in file:
                key, value = line.strip().split(': ')
                credentials[key] = value
        return credentials
    except FileNotFoundError:
        print(f"Error: '{file_path}' not found.")
        sys.exit(1)
    except ValueError:
        print(f"Error: Invalid format in '{file_path}'.")
        sys.exit(1)

# Helper function to convert hex to RGB
def hex_to_rgb(hex_code):
    hex_code = hex_code.lstrip('#')
    return tuple(int(hex_code[i:i+2], 16) for i in (0, 2, 4))

# Function to map hex codes to Miro's predefined colors
def map_hex_to_miro_color(hex_code):
    miro_colors = {
        "gray": "#808080",
        "light_yellow": "#ffff99",
        "yellow": "#ffeb3b",
        "orange": "#ff9800",
        "light_green": "#ccff90",
        "green": "#4caf50",
        "dark_green": "#388e3c",
        "cyan": "#00bcd4",
        "light_pink": "#f8bbd0",
        "pink": "#e91e63",
        "violet": "#9c27b0",
        "red": "#f44336",
        "light_blue": "#81d4fa",
        "blue": "#2196f3",
        "dark_blue": "#1565c0",
        "black": "#000000"
    }

    target_rgb = hex_to_rgb(hex_code)
    min_distance = float('inf')
    closest_color_name = "light_yellow"  # Default color

    for color_name, miro_hex in miro_colors.items():
        miro_rgb = hex_to_rgb(miro_hex)
        distance = sum((src - dst) ** 2 for src, dst in zip(target_rgb, miro_rgb))

        if distance < min_distance:
            min_distance = distance
            closest_color_name = color_name

    return closest_color_name

def create_poster_frame(board_id, width, height, access_token):
    url = f"https://api.miro.com/v2/boards/{board_id}/frames"
    headers = {
        "accept": "application/json",
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }
    payload = {
        "data": {
            "title": "Poster Frame"
        },
        "geometry": {
            "height": height,
            "width": width
        },
        "position": {
            "x": 0,
            "y": 0
        }
    }
    response = requests.post(url, json=payload, headers=headers)
    frame_info = response.json()
    print("Poster Frame Creation Response:", frame_info)

    # Check if the response contains an 'id' before returning it
    if 'id' in frame_info:
        return frame_info['id']
    else:
        print("Error creating poster frame:", frame_info)
        sys.exit(1)

# Function to scale coordinates and adjust for note size and poster position
def scale_coordinates(x, y, image_width, image_height, frame_width, frame_height):
    # Scale the coordinates from the image to the frame size
    scaled_x = (x / image_width) * frame_width
    scaled_y = (y / image_height) * frame_height

    # Adjust coordinates relative to the frame's top-left corner
    # Assuming the frame is centered on the board, we need to offset the coordinates
    offset_x = frame_width / 2
    offset_y = frame_height / 2
    adjusted_x = scaled_x - offset_x
    adjusted_y = scaled_y - offset_y

    return adjusted_x, adjusted_y

# Function to calculate the relative position within the original image and
# transform the position to the frame's coordinate system on the Miro board

def transform_to_frame_coordinates(x, y, csv_width, csv_height, frame_width, frame_height, note_width, note_height):
    # Translate CSV coordinates from top-left origin to center origin
    translated_x = x - (csv_width / 2)
    translated_y = (csv_height / 2) - y

    # Adjust the translated coordinates to be relative to the top-left of the frame
    adjusted_x = translated_x + (frame_width / 2)
    adjusted_y = (frame_height / 2) - translated_y

    # Adjust for the sticky note size
    adjusted_x -= note_width / 2
    adjusted_y -= note_height / 2

    # Ensure the sticky note is within the frame boundaries
    adjusted_x = max(min(adjusted_x, frame_width - note_width), 0)
    adjusted_y = max(min(adjusted_y, frame_height - note_height), 0)

    return adjusted_x, adjusted_y


# Modify the create_square_sticky_note function to accept the frame dimensions and use the new transform function

def create_square_sticky_note(board_id, text, x, y, hex_code, access_token, include_color, parent_id):
    url = f"https://api.miro.com/v2/boards/{board_id}/sticky_notes"
    headers = {
        "accept": "application/json",
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }

    # Determine the shape and set the geometry accordingly
    if NOTE_WIDTH == NOTE_HEIGHT:
        # If the note is square, only pass the height
        shape = "square"
        geometry = {"height": NOTE_HEIGHT}
    else:
        # If the note is rectangular, pass both width and height
        shape = "rectangle"
        geometry = {"width": NOTE_WIDTH, "height": NOTE_HEIGHT}

    payload = {
        "data": {
            "content": text,
            "shape": shape
        },
        "geometry": geometry,
        "parent": {
            "id": parent_id
        },
        "position": {
            "x": x,
            "y": y
        }
    }

    # Include color if specified
    if include_color:
        fillColor = map_hex_to_miro_color(hex_code)
        payload["style"] = {"fillColor": fillColor}

    response = requests.post(url, headers=headers, json=payload)
    print("Sticky Note Creation Response:", response.json())
    return response.json()


# Main function to process CSV and create notes on Miro
def process_csv_and_create_notes(csv_filename, board_id, access_token, include_color, parent_id):
    # Open the CSV file
    with open(csv_filename, mode='r', encoding='utf-8-sig') as file:
        # Initialize the CSV reader
        reader = csv.DictReader(file)

        # Iterate over each row in the CSV
        for row in reader:
            # Convert CSV coordinates to Miro frame coordinates
            x, y = transform_to_frame_coordinates(
                int(row['center_x']), int(row['center_y']),
                csv_width=3024, csv_height=4032,  # Original image size
                frame_width=3024, frame_height=4032,  # Frame size
                note_width=NOTE_WIDTH, note_height=NOTE_HEIGHT  # Sticky note dimensions
            )

            # Log the transformed coordinates for debugging
            print(f"Transformed coordinates for {row['extracted_text']}: x={x}, y={y}")

            # Create the sticky note
            create_square_sticky_note(
                board_id, row['extracted_text'], 
                x, y, row['hex_code'], 
                access_token, 
                include_color, 
                parent_id  # The ID of the poster frame
            )



# Entry point for the script
if __name__ == '__main__':
    # Default poster dimensions
    poster_width = 3024
    poster_height = 4032

    if len(sys.argv) < 2:
        print("Usage: python script.py <csv_filename> [--no-color] [poster-x-dimension] [poster-y-dimension]")
        sys.exit(1)

    csv_filename = sys.argv[1]
    include_color = '--no-color' not in sys.argv

    # Check for poster dimensions in arguments
    if len(sys.argv) > 3:
        poster_width = int(sys.argv[3])
    if len(sys.argv) > 4:
        poster_height = int(sys.argv[4])

    credentials = read_miro_credentials()
    access_token = credentials.get('access_token', '')
    board_id = credentials.get('board_id', '')
    if not access_token or not board_id:
        print("Error: Access token or board ID not found in 'miro_api_key.txt'.")
        sys.exit(1)

    clear()
    print("Creating poster frame on Miro board...")
    poster_id = create_poster_frame(board_id, poster_width, poster_height, access_token)

    print("Processing CSV and creating square notes on Miro board...")
    process_csv_and_create_notes(csv_filename, board_id, access_token, include_color, poster_id)
    print("Done.")